In [1]:
import cv2
import numpy as np
import mediapipe as mp

# Initialize Mediapipe Hands model
mp_hands = mp.solutions.hands
hands = mp_hands.Hands(max_num_hands=1, min_detection_confidence=0.7)

# Initialize drawing canvas
canvas = np.zeros((480, 640, 3), dtype=np.uint8)

# Last position for drawing
last_x, last_y = None, None

# Function to process the frame and detect hand landmarks
def process_frame(frame):
    rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    results = hands.process(rgb_frame)
    return results

# Function to extract the positions of the fingers
def extract_landmarks(results):
    if results.multi_hand_landmarks:
        landmarks = []
        for hand_landmarks in results.multi_hand_landmarks:
            for lm in hand_landmarks.landmark:
                landmarks.append((int(lm.x * 640), int(lm.y * 480)))  # Scale to frame size
        return landmarks
    return None

# Function to enhance the video quality
def enhance_video(frame):
    # Increase brightness and contrast
    alpha = 1.5  # Contrast control (1.0-3.0)
    beta = 30    # Brightness control (0-100)
    enhanced_frame = cv2.convertScaleAbs(frame, alpha=alpha, beta=beta)

    # Optionally apply Gaussian Blur to reduce noise
    enhanced_frame = cv2.GaussianBlur(enhanced_frame, (5, 5), 0)

    return enhanced_frame

# Main function for virtual mouse and drawing
def virtual_mouse_drawing():
    global last_x, last_y
    cap = cv2.VideoCapture(0)

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break
        
        # Flip the frame horizontally to remove mirror effect
        frame = cv2.flip(frame, 1)

        # Enhance video quality
        enhanced_frame = enhance_video(frame)

        # Process the frame to detect hand landmarks
        results = process_frame(enhanced_frame)
        landmarks = extract_landmarks(results)

        if landmarks:
            index_finger = landmarks[mp_hands.HandLandmark.INDEX_FINGER_TIP]
            middle_finger = landmarks[mp_hands.HandLandmark.MIDDLE_FINGER_TIP]

            # Draw cursor at index finger tip position
            cursor_x, cursor_y = index_finger
            cv2.circle(enhanced_frame, (cursor_x, cursor_y), 10, (0, 255, 0), -1)  # Draw cursor

            # Determine if we are drawing or ignoring
            if middle_finger[1] < cursor_y:  # Middle finger above index finger
                if last_x is not None and last_y is not None:
                    cv2.line(canvas, (last_x, last_y), (cursor_x, cursor_y), (255, 255, 255), 5)
                last_x, last_y = cursor_x, cursor_y  # Update last position for drawing
            else:
                last_x, last_y = None, None  # Reset last position to stop drawing

        # Display the canvas
        combined_frame = cv2.addWeighted(enhanced_frame, 0.5, canvas, 0.5, 0)
        cv2.imshow('Virtual Mouse and Drawing Board', combined_frame)

        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()

# Start the virtual mouse and drawing functionality
if __name__ == "__main__":
    virtual_mouse_drawing()


ModuleNotFoundError: No module named 'cv2'

In [1]:
import cv2
import numpy as np
import mediapipe as mp
import pytesseract
import sympy as sp

# Initialize Mediapipe Hands model
mp_hands = mp.solutions.hands
hands = mp_hands.Hands(max_num_hands=1, min_detection_confidence=0.7)

# Initialize drawing canvas
canvas = np.zeros((480, 640, 3), dtype=np.uint8)

# Last position for drawing
last_x, last_y = None, None

# Variables to store recognized equation and solution
recognized_equation = ""
solution = ""

# Function to process the frame and detect hand landmarks
def process_frame(frame):
    rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    results = hands.process(rgb_frame)
    return results

# Function to extract the positions of the fingers
def extract_landmarks(results):
    if results.multi_hand_landmarks:
        landmarks = []
        for hand_landmarks in results.multi_hand_landmarks:
            for lm in hand_landmarks.landmark:
                landmarks.append((int(lm.x * 640), int(lm.y * 480)))  # Scale to frame size
        return landmarks
    return None

# Function to enhance the video quality
def enhance_video(frame):
    alpha = 1.5  # Contrast control (1.0-3.0)
    beta = 30    # Brightness control (0-100)
    enhanced_frame = cv2.convertScaleAbs(frame, alpha=alpha, beta=beta)
    enhanced_frame = cv2.GaussianBlur(enhanced_frame, (5, 5), 0)
    return enhanced_frame

# Function to recognize and solve the drawn equation
def recognize_and_solve_equation(canvas):
    # Convert the canvas to grayscale for OCR
    gray_canvas = cv2.cvtColor(canvas, cv2.COLOR_BGR2GRAY)
    _, thresh = cv2.threshold(gray_canvas, 150, 255, cv2.THRESH_BINARY_INV)

    # Use pytesseract to extract text from the image
    equation = pytesseract.image_to_string(thresh, config='--psm 6')
    equation = equation.strip()  # Clean up the extracted string

    if equation:
        try:
            # Solve the equation using sympy
            x = sp.symbols('x')
            solution = sp.solve(equation, x)
            return equation, solution
        except Exception as e:
            return equation, f"Error solving: {e}"

    return None, None

# Main function for virtual mouse and drawing
def virtual_mouse_drawing():
    global last_x, last_y, recognized_equation, solution
    cap = cv2.VideoCapture(0)

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break
        
        # Flip the frame horizontally to remove mirror effect
        frame = cv2.flip(frame, 1)

        # Enhance video quality
        enhanced_frame = enhance_video(frame)

        # Process the frame to detect hand landmarks
        results = process_frame(enhanced_frame)
        landmarks = extract_landmarks(results)

        if landmarks:
            index_finger = landmarks[mp_hands.HandLandmark.INDEX_FINGER_TIP]
            middle_finger = landmarks[mp_hands.HandLandmark.MIDDLE_FINGER_TIP]

            # Draw cursor at index finger tip position
            cursor_x, cursor_y = index_finger
            cv2.circle(enhanced_frame, (cursor_x, cursor_y), 10, (0, 255, 0), -1)  # Draw cursor

            # Determine if we are drawing or ignoring
            if middle_finger[1] < cursor_y:  # Middle finger above index finger
                if last_x is not None and last_y is not None:
                    cv2.line(canvas, (last_x, last_y), (cursor_x, cursor_y), (255, 255, 255), 5)
                last_x, last_y = cursor_x, cursor_y  # Update last position for drawing
            else:
                last_x, last_y = None, None  # Reset last position to stop drawing

        # Display the canvas
        combined_frame = cv2.addWeighted(enhanced_frame, 0.5, canvas, 0.5, 0)

        # Display recognized equation and its solution on the frame
        if recognized_equation:
            cv2.putText(combined_frame, f"Equation: {recognized_equation}", (10, 460), 
                        cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 255, 255), 2)
            cv2.putText(combined_frame, f"Solution: {solution}", (10, 480), 
                        cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 255, 255), 2)

        cv2.imshow('Virtual Mouse and Drawing Board', combined_frame)

        # Check if the user wants to solve the equation
        if cv2.waitKey(1) & 0xFF == ord('s'):  # Press 's' to solve the equation
            recognized_equation, solution = recognize_and_solve_equation(canvas)
            if recognized_equation:
                print(f"Recognized Equation: {recognized_equation}")
                print(f"Solution: {solution}")
            else:
                recognized_equation = "No equation recognized."

        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()

# Start the virtual mouse and drawing functionality
if __name__ == "__main__":
    virtual_mouse_drawing()


C:\Users\user\AppData\Roaming\Python\Python310\site-packages\google\protobuf\symbol_database.py:55: UserWarning: SymbolDatabase.GetPrototype() is deprecated. Please use message_factory.GetMessageClass() instead. SymbolDatabase.GetPrototype() will be removed soon.
  warnings.warn('SymbolDatabase.GetPrototype() is deprecated. Please '


KeyboardInterrupt: 